In [1]:
import configparser
config = configparser.ConfigParser()
config.read('config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')

users=config.get('Configuration', 'users')
users = [(item.strip()) for item in users.split(',')]

scenes=config.get('Configuration', 'scenes')
scenes = [(item.strip()) for item in scenes.split(',')]

room=config.get('Configuration', 'room')

In [2]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")


t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

days=list(range(day_time(t0),day_time(tN)+1))
print(days)

2024-03-16 21:31:18 Init date is 1710621078 day: 19798 2024-03-16 21:31:18
2024-03-16 21:41:04 End date is 1710621664 day: 19798 2024-03-16 21:41:04
[19798]


In [3]:
def getLoc(room,s):
    print("S:",s)
    xy=config.get(room, s)
    xy = [(item.strip()) for item in xy.split(',')]
    xy = [list(map(int, item.split('x'))) for item in xy]
    return xy

loc_sensor={}

fp_sensors = config.get(room, 'maps')
fp_sensors = [(item.strip()) for item in fp_sensors.split(',')]

w = int(config.get(room, 'w'))
h = int(config.get(room, 'h'))

ix = int(config.get(room, 'ix'))
lx = int(config.get(room, 'lx'))
iy = int(config.get(room, 'iy'))
ly = int(config.get(room, 'ly'))



In [4]:
df_total_map = {}
scene_user = {}
for d in days:
    df_total_map[d]= {}
    for user in users:
        df_total_map[d][user]=None
        scene_user[user] = {}
        for s in scenes:
            print(d,source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv")
            try:
                series=read_csv(source_folder+"/"+s+"/location_gt_"+user+"/"+str(d)+".location.tsv", sep="\t",parse_dates=True, header=None, names=["time","date","time2","x","y"])
                print(d,series.size,(series["time"].iloc[0],series["time"].iloc[-1]))
                scene_user[user][s] = (series["time"].iloc[0],series["time"].iloc[-1])
                print(scene_user[user][s])
            except Exception as e:
                print("Not data",e)
                continue
            if(df_total_map[d][user] is None):
                df_total_map[d][user]=series
            else:
                df_total_map[d][user] = df_total_map[d][user].append(series, ignore_index=True)


19798 ./data/kitchen/2-person-labelled/Scene1/location_gt_ed9c/19798.location.tsv
19798 1215 (1710621088, 1710621359)
(1710621088, 1710621359)
19798 ./data/kitchen/2-person-labelled/Scene2/location_gt_ed9c/19798.location.tsv
19798 1380 (1710621360, 1710621664)
(1710621360, 1710621664)
19798 ./data/kitchen/2-person-labelled/Scene1/location_gt_867f/19798.location.tsv
19798 1145 (1710621078, 1710621359)
(1710621078, 1710621359)
19798 ./data/kitchen/2-person-labelled/Scene2/location_gt_867f/19798.location.tsv
19798 1255 (1710621362, 1710621664)
(1710621362, 1710621664)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_28572\2636859251.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][user] = df_total_map[d][user].append(series, ignore_index=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_28572\2636859251.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_total_map[d][user] = df_total_map[d][user].append(series, ignore_index=True)


In [5]:
def getScene(t0):
    for user, user_scene in scene_user.items():
        for scene, times in user_scene.items():
            print(t0,times[0],times[1])
            if(t0>times[0]-window_size/2 and t0<times[1]+window_size/2):
                return scene
    return None

In [6]:
def getXY(x,y):
    x0=int(x*w/ix)
    y0=int(y*h/iy)
    if(x0>=w):
        x0=w-1
    if(y0>=h):
        y0=h-1
    return y0,x0

In [7]:
import cv2
import os

last_df_intervalo=None
for d in days:
    print(d)
    os.makedirs(data_folder+"/"+room+"/DAY_"+str(d)+"/", exist_ok=True)
    for user in users:
        df_total=df_total_map[d][user]
        for tt in range(t0,tN,1):
            print(tt,day_time(tt))
            if(day_time(tt) != d):
                continue
                
            scene=getScene(tt)
            print("Scene",scene)
            if not scene:
                last_df_intervalo=None
                continue
            img0=np.zeros((h, w))
            df_intervalo = df_total[(df_total['time'] >= (tt-window_size/2)) & (df_total['time'] <= (tt))]
            print("DF",df_intervalo)     
            if df_intervalo.empty:
                print("Zero GT complete with last interval",df_intervalo)   
                df_intervalo=last_df_intervalo
            else:
                last_df_intervalo=df_intervalo
            if df_intervalo is None or df_intervalo.empty:
                continue
            for index, row in df_intervalo.iterrows():
                dif=abs(tt-row['time'])+1
                print("@TIME",tt,row, row['time'], "dif:", dif, "xy:", row['x'],row['y'], getXY(row['x'],row['y']))
                xy=getXY(row['x'],row['y'])
                img0[xy[0]][xy[1]]=img0[xy[0]][xy[1]]+int(255/dif)
                img0[img0>255]=255
            cv2.imwrite(data_folder+"/"+room+"/DAY_"+str(d)+"/GT."+user+"."+str(tt)+".png", img0) 

19798
1710621078 19798
1710621078 1710621088 1710621359
1710621078 1710621360 1710621664
1710621078 1710621078 1710621359
Scene Scene1
DF Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
Zero GT complete with last interval Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
1710621079 19798
1710621079 1710621088 1710621359
1710621079 1710621360 1710621664
1710621079 1710621078 1710621359
Scene Scene1
DF Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
Zero GT complete with last interval Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
1710621080 19798
1710621080 1710621088 1710621359
1710621080 1710621360 1710621664
1710621080 1710621078 1710621359
Scene Scene1
DF Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
Zero GT complete with last interval Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
1710621081 19798
1710621081 1710621088 1710621359
1710621081 1710621360 1710621664
1710621081 1710621078 1710621359
Scene Sc

1710621143 19798
1710621143 1710621088 1710621359
Scene Scene1
DF           time                 date       time2    x    y
56  1710621142  2024-03-16 21:32:22  1710621142  160  155
57  1710621142  2024-03-16 21:32:22  1710621142  156  156
@TIME 1710621143 time              1710621142
date     2024-03-16 21:32:22
time2             1710621142
x                        160
y                        155
Name: 56, dtype: object 1710621142 dif: 2 xy: 160 155 (2, 2)
@TIME 1710621143 time              1710621142
date     2024-03-16 21:32:22
time2             1710621142
x                        156
y                        156
Name: 57, dtype: object 1710621142 dif: 2 xy: 156 156 (2, 2)
1710621144 19798
1710621144 1710621088 1710621359
Scene Scene1
DF           time                 date       time2    x    y
56  1710621142  2024-03-16 21:32:22  1710621142  160  155
57  1710621142  2024-03-16 21:32:22  1710621142  156  156
58  1710621144  2024-03-16 21:32:24  1710621144  162  161
59  1710621144  

1710621167 19798
1710621167 1710621088 1710621359
Scene Scene1
DF           time                 date       time2    x    y
79  1710621165  2024-03-16 21:32:45  1710621165  158  159
80  1710621167  2024-03-16 21:32:47  1710621167  160  159
@TIME 1710621167 time              1710621165
date     2024-03-16 21:32:45
time2             1710621165
x                        158
y                        159
Name: 79, dtype: object 1710621165 dif: 3 xy: 158 159 (2, 2)
@TIME 1710621167 time              1710621167
date     2024-03-16 21:32:47
time2             1710621167
x                        160
y                        159
Name: 80, dtype: object 1710621167 dif: 1 xy: 160 159 (2, 2)
1710621168 19798
1710621168 1710621088 1710621359
Scene Scene1
DF           time                 date       time2    x    y
80  1710621167  2024-03-16 21:32:47  1710621167  160  159
81  1710621168  2024-03-16 21:32:48  1710621168  158  161
@TIME 1710621168 time              1710621167
date     2024-03-16 21:32:47

1710621230 19798
1710621230 1710621088 1710621359
Scene Scene1
DF            time                 date       time2    x    y
139  1710621228  2024-03-16 21:33:48  1710621228  155  168
@TIME 1710621230 time              1710621228
date     2024-03-16 21:33:48
time2             1710621228
x                        155
y                        168
Name: 139, dtype: object 1710621228 dif: 3 xy: 155 168 (2, 2)
1710621231 19798
1710621231 1710621088 1710621359
Scene Scene1
DF Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
Zero GT complete with last interval Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
@TIME 1710621231 time              1710621228
date     2024-03-16 21:33:48
time2             1710621228
x                        155
y                        168
Name: 139, dtype: object 1710621228 dif: 4 xy: 155 168 (2, 2)
1710621232 19798
1710621232 1710621088 1710621359
Scene Scene1
DF Empty DataFrame
Columns: [time, date, time2, x, y]
Index: []
Zero GT complete 

1710621261 19798
1710621261 1710621088 1710621359
Scene Scene1
DF            time                 date       time2    x    y
159  1710621259  2024-03-16 21:34:19  1710621259  323  165
160  1710621260  2024-03-16 21:34:20  1710621260  321  162
161  1710621261  2024-03-16 21:34:21  1710621261  328  162
@TIME 1710621261 time              1710621259
date     2024-03-16 21:34:19
time2             1710621259
x                        323
y                        165
Name: 159, dtype: object 1710621259 dif: 3 xy: 323 165 (2, 4)
@TIME 1710621261 time              1710621260
date     2024-03-16 21:34:20
time2             1710621260
x                        321
y                        162
Name: 160, dtype: object 1710621260 dif: 2 xy: 321 162 (2, 4)
@TIME 1710621261 time              1710621261
date     2024-03-16 21:34:21
time2             1710621261
x                        328
y                        162
Name: 161, dtype: object 1710621261 dif: 1 xy: 328 162 (2, 4)
1710621262 19798
171062126

...

1710621451 19798
1710621451 1710621088 1710621359
1710621451 1710621360 1710621664
Scene Scene2
DF            time                 date       time2    x   y
305  1710621449  2024-03-16 21:37:29  1710621449  184  68
306  1710621450  2024-03-16 21:37:30  1710621450  170  74
@TIME 1710621451 time              1710621449
date     2024-03-16 21:37:29
time2             1710621449
x                        184
y                         68
Name: 305, dtype: object 1710621449 dif: 3 xy: 184 68 (0, 2)
@TIME 1710621451 time              1710621450
date     2024-03-16 21:37:30
time2             1710621450
x                        170
y                         74
Name: 306, dtype: object 1710621450 dif: 2 xy: 170 74 (0, 2)
1710621452 19798
1710621452 1710621088 1710621359
1710621452 1710621360 1710621664
Scene Scene2
DF            time                 date       time2    x   y
306  1710621450  2024-03-16 21:37:30  1710621450  170  74
@TIME 1710621452 time              1710621450
date     2024-03-16 

1710621489 19798
1710621489 1710621088 1710621359
1710621489 1710621360 1710621664
Scene Scene2
DF            time                 date       time2    x    y
343  1710621487  2024-03-16 21:38:07  1710621487  316  430
344  1710621487  2024-03-16 21:38:07  1710621487  322  420
345  1710621488  2024-03-16 21:38:08  1710621488  322  420
346  1710621489  2024-03-16 21:38:09  1710621489  322  420
@TIME 1710621489 time              1710621487
date     2024-03-16 21:38:07
time2             1710621487
x                        316
y                        430
Name: 343, dtype: object 1710621487 dif: 3 xy: 316 430 (5, 4)
@TIME 1710621489 time              1710621487
date     2024-03-16 21:38:07
time2             1710621487
x                        322
y                        420
Name: 344, dtype: object 1710621487 dif: 3 xy: 322 420 (5, 4)
@TIME 1710621489 time              1710621488
date     2024-03-16 21:38:08
time2             1710621488
x                        322
y                        

1710621543 19798
1710621543 1710621088 1710621359
1710621543 1710621360 1710621664
Scene Scene2
DF            time                 date       time2    x    y
384  1710621541  2024-03-16 21:39:01  1710621541  371  263
385  1710621542  2024-03-16 21:39:02  1710621542  371  268
@TIME 1710621543 time              1710621541
date     2024-03-16 21:39:01
time2             1710621541
x                        371
y                        263
Name: 384, dtype: object 1710621541 dif: 3 xy: 371 263 (3, 4)
@TIME 1710621543 time              1710621542
date     2024-03-16 21:39:02
time2             1710621542
x                        371
y                        268
Name: 385, dtype: object 1710621542 dif: 2 xy: 371 268 (3, 4)
1710621544 19798
1710621544 1710621088 1710621359
1710621544 1710621360 1710621664
Scene Scene2
DF            time                 date       time2    x    y
385  1710621542  2024-03-16 21:39:02  1710621542  371  268
386  1710621544  2024-03-16 21:39:04  1710621544  371  268


1710621580 19798
1710621580 1710621088 1710621359
1710621580 1710621360 1710621664
Scene Scene2
DF            time                 date       time2    x    y
409  1710621578  2024-03-16 21:39:38  1710621578  352  186
410  1710621579  2024-03-16 21:39:39  1710621579  343  169
411  1710621580  2024-03-16 21:39:40  1710621580  338  155
412  1710621580  2024-03-16 21:39:40  1710621580  336  146
@TIME 1710621580 time              1710621578
date     2024-03-16 21:39:38
time2             1710621578
x                        352
y                        186
Name: 409, dtype: object 1710621578 dif: 3 xy: 352 186 (2, 4)
@TIME 1710621580 time              1710621579
date     2024-03-16 21:39:39
time2             1710621579
x                        343
y                        169
Name: 410, dtype: object 1710621579 dif: 2 xy: 343 169 (2, 4)
@TIME 1710621580 time              1710621580
date     2024-03-16 21:39:40
time2             1710621580
x                        338
y                        

1710621631 19798
1710621631 1710621088 1710621359
1710621631 1710621360 1710621664
Scene Scene2
DF            time                 date       time2    x    y
460  1710621629  2024-03-16 21:40:29  1710621629  338  263
461  1710621630  2024-03-16 21:40:30  1710621630  338  263
462  1710621630  2024-03-16 21:40:30  1710621630  338  263
463  1710621630  2024-03-16 21:40:30  1710621630  334  283
@TIME 1710621631 time              1710621629
date     2024-03-16 21:40:29
time2             1710621629
x                        338
y                        263
Name: 460, dtype: object 1710621629 dif: 3 xy: 338 263 (3, 4)
@TIME 1710621631 time              1710621630
date     2024-03-16 21:40:30
time2             1710621630
x                        338
y                        263
Name: 461, dtype: object 1710621630 dif: 2 xy: 338 263 (3, 4)
@TIME 1710621631 time              1710621630
date     2024-03-16 21:40:30
time2             1710621630
x                        338
y                        